In [4]:
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from torch import nn
from torch.optim import Adam
from tqdm import tqdm

In [5]:
df = pd.read_csv('../../dataframe.csv')
df = df.dropna(subset=['text', 'is_offensive'])
df['is_offensive'] = df['is_offensive'].astype('int')
df['target'] = df['target'].map({'INSULT': 0, 'RACIST': 1, 'SEXIST': 2, 'PROFANITY': 3, 'OTHER': 4})
df = df[~(((df['is_offensive'] == 0) & (df['target'] != 4)) | df['text'].isnull() | (df['is_offensive'] == 1) & (df['target'] == 4))]
df = df.drop(['id','target'], axis=1)
df.reset_index(drop=True, inplace=True)
for i in range(len(df)):
    if len(df['text'][i]) == 1:
        #delete row
        df = df.drop(i)
df.reset_index(drop=True, inplace=True)
df.head()

,text,is_offensive
0,çürük dişli,1
1,Bu adamın islama ve müslümanlara verdiği zarar...,1
2,erkekler zora gelmez,1
3,Utanmazın götüne kazık sokmuşlar bu tıkırtı ne...,1
4,otomasyon< sistemlerine= doğrudan bağlanabilir,0


In [6]:
tokenizer = BertTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")

class Dataset(torch.utils.data.Dataset):
    
    def __init__(self, df):

        self.labels = [label for label in df['is_offensive']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['text']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [7]:
np.random.seed(42)
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42), 
                                     [int(.8*len(df)), int(.9*len(df))])

print(len(df_train),len(df_val), len(df_test))

9928 1241 1242


In [8]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained("dbmdz/bert-base-turkish-cased")
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 2)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask, return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [9]:
def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:
        model = model.cuda()
        criterion = criterion.cuda()

    for epoch_num in range(epochs):

        total_acc_train = 0
        total_loss_train = 0

        for train_input, train_label in tqdm(train_dataloader):

            train_label = train_label.to(device)
            mask = train_input['attention_mask'].to(device)
            input_id = train_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask)
            
            batch_loss = criterion(output, train_label.long())
            total_loss_train += batch_loss.item()
            
            acc = (output.argmax(dim=1) == train_label).sum().item()
            total_acc_train += acc

            model.zero_grad()
            batch_loss.backward()
            optimizer.step()
        
        total_acc_val = 0
        total_loss_val = 0

        with torch.no_grad():

            for val_input, val_label in val_dataloader:

                val_label = val_label.to(device)
                mask = val_input['attention_mask'].to(device)
                input_id = val_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)

                batch_loss = criterion(output, val_label.long())
                total_loss_val += batch_loss.item()
                
                acc = (output.argmax(dim=1) == val_label).sum().item()
                total_acc_val += acc
        
        print(
            f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
            | Train Accuracy: {total_acc_train / len(train_data): .3f} \
            | Val Loss: {total_loss_val / len(val_data): .3f} \
            | Val Accuracy: {total_acc_val / len(val_data): .3f}')
                  
EPOCHS = 2
model = BertClassifier()
LR = 1e-6
              
train(model, df_train, df_val, LR, EPOCHS)

Some weights of the model checkpoint at dbmdz/bert-base-turkish-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 4964/4964 [35:32<00:00,  2.33it/s]


Epochs: 1 | Train Loss:  0.146             | Train Accuracy:  0.874             | Val Loss:  0.067             | Val Accuracy:  0.952


100%|██████████| 4964/4964 [35:28<00:00,  2.33it/s]


Epochs: 2 | Train Loss:  0.045             | Train Accuracy:  0.970             | Val Loss:  0.051             | Val Accuracy:  0.969


In [13]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    
evaluate(model, df_test)

Test Accuracy:  0.962


In [14]:
# try to predict a sentence
def predict(model, sentence):
    
        use_cuda = torch.cuda.is_available()
        device = torch.device("cuda" if use_cuda else "cpu")
    
        if use_cuda:
            model = model.cuda()
    
        with torch.no_grad():
    
            input_id = tokenizer(sentence, padding='max_length', max_length = 512, truncation=True, return_tensors="pt")
            mask = input_id['attention_mask'].to(device)
            input_id = input_id['input_ids'].squeeze(1).to(device)
    
            output = model(input_id, mask)
    
            return output.argmax(dim=1).item()

In [26]:
predict(model, "yarınki belgesel programını dört gözle bekliyorum")

0

In [31]:
df_test['text'].iloc[0]

'gürcülerin ne kadar kaba olduğunu bilirim'

In [32]:
true = 0
for i in range(len(df_test.text)):
    result = predict(model, df_test.text.iloc[i])
    if result == df_test.is_offensive.iloc[i]:
        true += 1

acc = true / len(df_test.text)

In [33]:
acc

0.9589371980676329

In [34]:
df_test['is_offensive'].value_counts()

1    891
0    351
Name: is_offensive, dtype: int64

In [39]:
torch.save(model, './models/bert_model_is_offensive.pt')

In [41]:
from sklearn.metrics import f1_score

predicted = []
for i in range(len(df_test.text)):
    predicted.append(predict(model, df_test.text.iloc[i]))
    
f1_score(df_test.is_offensive, predicted)

0.9736694677871149